In [1]:
import random
import os
import numpy as np
import pandas as pd
from datetime import datetime
from autogluon.tabular import TabularPredictor

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)

# ==========================================
# 1. 데이터 로드
# ==========================================

train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
target = '임신 성공 여부'

# ==========================================
# 2. 파생변수 생성
# ==========================================

def derive_features(df):

    age_order = {'만18-34세': 1, '만35-37세': 2, '만38-39세': 3, '만40-42세': 4, '만43-44세': 5, '만45-50세': 6, '알 수 없음': 0}
    df['나이_순서'] = df['시술 당시 나이'].map(age_order)
    df['나이x배아'] = df['나이_순서'] * df['이식된 배아 수']
    df['연령별 배아 효율'] = df['이식된 배아 수'] / (df['나이_순서'] + 1e-5) # '나이가 적으면서 배아가 많은 그룹' vs '나이가 많으면서 배아가 적은 그룹' 강조

    df['배아 발달 기간'] = df['배아 이식 경과일'] - df['난자 혼합 경과일']
    df['이식 비중'] = df['이식된 배아 수'] / (df['이식된 배아 수'] + df['저장된 배아 수'] + 1e-5)

    oocyte_cols = ['수집된 신선 난자 수', '혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '해동 난자 수']
    df['총 난자 수'] = df[oocyte_cols].fillna(0).sum(axis=1)
    df['배아 손실률'] = (df['총 난자 수'] - df['총 생성 배아 수']) / df['총 난자 수']
    
    df['배아 생성 효율'] = df['저장된 배아 수'] / (df['저장된 신선 난자 수'] + 1e-6)
    df['수정 효율'] = df['총 생성 배아 수'] / (df['혼합된 난자 수'] + 1e-6)
    df['선별 효율'] = df['저장된 배아 수'] / (df['총 생성 배아 수'] + 1e-6)
    
    def categorize_age_optimized(age_label):
        if age_label in ['만35-37세', '만38-39세']:
            return 1  # Middle
        elif age_label in ['만40-42세', '만43-44세', '만45-50세']:
            return 2  # Senior
        else:
            return 0  # '만18-34세'와 '알 수 없음'을 모두 0으로 처리
        
    df['나이_세분화'] = df['시술 당시 나이'].apply(categorize_age_optimized)

    def clean_treatment(text): 
        text = str(text).upper()
        if 'ICSI' in text:
            return 'ICSI'
        if 'IVF' in text:
            return 'IVF'
        if 'IUI' in text:
            return 'IUI'
        return 'Other'

    df['시술유형_정제'] = df['특정 시술 유형'].apply(clean_treatment)
    
    drop_cols = ['특정 시술 유형', '시술 유형', 'DI 출산 횟수', 'DI 임신 횟수','대리모 여부', '난자 해동 경과일', '배아 해동 경과일',
                 '난자 기증자 나이', '저장된 신선 난자 수', 'PGD 시술 여부', '난자 혼합 경과일',
                 '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '불임 원인 - 자궁경부 문제', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
                '부부 주 불임 원인', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '부부 부 불임 원인', '여성 부 불임 원인']
    df.drop(drop_cols, axis=1, inplace=True)
    
    return df

train_df = derive_features(train_df)
test_df = derive_features(test_df)

# ==========================================
# 3. 모델 학습 설정
# ==========================================

# 불균형 비율 계산 (190123 / 66228 = 약 2.87)
scale_weight = 190123 / 66228

custom_hyperparameters = {
    'GBM': [
        {'is_unbalance': True, 'ag_args': {'name_suffix': '_Unbal'}}, # LGBM 전용 불균형 옵션
        {'scale_pos_weight': scale_weight, 'learning_rate': 0.03, 'ag_args': {'name_suffix': '_Weighted'}},
        {'extra_trees': True, 'ag_args': {'name_suffix': '_XT'}} # 변동성 부여
    ],
    'XGB': [
        {'scale_pos_weight': scale_weight, 'max_depth': 6, 'ag_args': {'name_suffix': '_Weighted'}},
        {'max_depth': 8, 'learning_rate': 0.01, 'ag_args': {'name_suffix': '_Deep'}}
    ],
    'NN_TORCH': [
        {'weight_estimation': True}, # NeuralNet 불균형 가중치
        {'activation': 'elu', 'dropout_prob': 0.15}
    ]
}

predictor = TabularPredictor(
    label=target, 
    eval_metric='roc_auc',
    path='ag_models_out',
).fit(
    train_data=train_df,
    time_limit=3600 * 3,
    presets='best_quality',
    ag_args_fit={'num_gpus': 1},
    included_model_types=['GBM', 'CAT', 'XGB', 'RF', 'XT', 'NN_TORCH'],
    num_stack_levels=2,
    num_bag_folds=5,
    refit_full=True
)

# ==========================================
# 4. 예측 (Test Data 활용) - 최종 결과를 확률로 출력 (Positive 클래스에 대한 확률만 추출)
# ==========================================

pred_probs = predictor.predict_proba(test_df)
final_probs = pred_probs.iloc[:, 1]


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:40 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6000
CPU Count:          10
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       3.24 GB / 16.00 GB (20.3%)
Disk Space Avail:   187.75 GB / 460.43 GB (40.8%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether 

In [2]:
# --- 1. 리더보드 (오름차순 정렬) ---
lb = predictor.leaderboard(silent=True)
display(lb.sort_values(by='score_val', ascending=False))

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.740336,roc_auc,22.282046,7179.077192,0.033559,11.462259,2,True,15
1,XGBoost_r89_BAG_L1,0.739357,roc_auc,1.580842,133.542065,1.580842,133.542065,1,True,12
2,XGBoost_BAG_L1,0.738910,roc_auc,1.525679,98.548495,1.525679,98.548495,1,True,5
3,XGBoost_r194_BAG_L1,0.738903,roc_auc,0.545960,29.285895,0.545960,29.285895,1,True,14
4,NeuralNetTorch_r79_BAG_L1,0.737879,roc_auc,4.864224,2356.947834,4.864224,2356.947834,1,True,7
5,NeuralNetTorch_BAG_L1,0.737007,roc_auc,4.299485,2127.915156,4.299485,2127.915156,1,True,6
6,XGBoost_r33_BAG_L1,0.736865,roc_auc,4.832037,267.196338,4.832037,267.196338,1,True,9
7,NeuralNetTorch_r22_BAG_L1,0.735985,roc_auc,3.758694,2410.661886,3.758694,2410.661886,1,True,8
8,ExtraTreesEntr_BAG_L1,0.732092,roc_auc,5.120908,7.501380,5.120908,7.501380,1,True,4
9,ExtraTreesGini_BAG_L1,0.731530,roc_auc,5.304253,7.761490,5.304253,7.761490,1,True,3


In [3]:
# --- 2. 피처 중요도 ---
fi = predictor.feature_importance(data=train_df.sample(n=min(5000, len(train_df)), random_state=42))
display(fi)

These features in provided data are not utilized by the predictor and will be ignored: ['ID', '불임 원인 - 여성 요인']
Computing feature importance via permutation shuffling for 56 features using 5000 rows with 5 shuffle sets...
	645.13s	= Expected runtime (129.03s per shuffle set)
	398.78s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
파트너 정자와 혼합된 난자 수,0.003495,0.000080,3.232168e-08,5,0.003659,0.003331
미세주입에서 생성된 배아 수,0.002947,0.000143,6.568078e-07,5,0.003240,0.002653
혼합된 난자 수,0.004235,0.000237,1.174966e-06,5,0.004724,0.003747
총 난자 수,0.005140,0.000296,1.310661e-06,5,0.005749,0.004531
불명확 불임 원인,0.001059,0.000066,1.835840e-06,5,0.001196,0.000923
미세주입된 난자 수,0.002985,0.000199,2.355443e-06,5,0.003394,0.002575
수집된 신선 난자 수,0.005338,0.000366,2.630219e-06,5,0.006092,0.004585
총 시술 횟수,0.003209,0.000240,3.728653e-06,5,0.003703,0.002715
수정 효율,0.005455,0.000446,5.330351e-06,5,0.006374,0.004536
미세주입 후 저장된 배아 수,0.000978,0.000080,5.365486e-06,5,0.001143,0.000813


In [5]:
# --- 3. 제출 파일 생성 ---
submission = pd.read_csv('../Data/sample_submission.csv')
submission['probability'] = final_probs.values

# 현재 시간 가져오기 (예: 0206_1031)
now = datetime.now().strftime('%m%d_%H%M')
file_name = f"{now}_submission.csv"
submission.to_csv(file_name, index=False)

print(f"학습 및 예측이 완료되었습니다. 결과가 {file_name}에 저장되었습니다.")

학습 및 예측이 완료되었습니다. 결과가 0206_2005_submission.csv에 저장되었습니다.
